## Introduction
- This Python workbook demonstrating how to identify top-selling SKUs and perform 80-20 (Pareto) analysis to uncover business-critical insights using an FMCG retail sales dataset.
- It covers file reading, data cleaning, Data Processing and sales analysis methods.


In [ ]:
# !pip install matplotlib
# !pip install seaborn

### Import Libraries
Step 1: Import necessary libraries for data manipulation and date operations


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import calendar

import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

### Data Loading and Exploration
Step 2: Read the sales dataset (CSV file) into a pandas DataFrame

- Data path - < Local Path>


In [ ]:
# Read CSV file
df = pd.read_csv("retail_sales_data_set1_2024.csv")

Step 3: Display the shape and column names to understand data structure

In [ ]:
# Shape of DataFrame (rows, columns)
print(df.shape)

# # Column names
# print(df.columns)

# Column names as a list
print(df.columns.tolist())

Step 4: Preview the first few rows of the dataset for a quick check

In [ ]:
# check the dataframe
df.head()

### Unique Values and Data Types
Step 5: Find unique Item Names, and check numbers of unique items/departments


In [ ]:
df['Item_Name'].unique().tolist()
df['Item_Name'].nunique(), df['Department'].nunique()

Step 6: Check data types for each column to verify correct format before further processing



In [ ]:
# Shows column data types
df.dtypes

Step 7: Get detailed info including missing values and memory usage

In [ ]:
##  shows types along with non-null counts.
df.info()

### Data Processing
Step 8: Convert date strings to datetime objects for easier date-based analysis

In [ ]:
df['Sales_RealDate'] = pd.to_datetime(df['Sales_RealDate'], errors='coerce')

Step 9: Create new features - month, year, month labels

In [ ]:
df['Month_of_purchase'] = df['Sales_RealDate'].dt.month
df['Year_of_purchase'] = df['Sales_RealDate'].dt.year
df['Month'] = df['Month_of_purchase'].apply(lambda x: calendar.month_abbr[x])
df['Month_year'] = df.Year_of_purchase.map(str)+ "-" + df.Month.map(str)


In [ ]:
df.head(5)

### Data Cleaning
Step 10: Detect and handle missing values across all columns


In [ ]:
df.isnull().sum()

Step 11: Fill missing Item Names and Department values and clean text data

In [ ]:
###Calculating Total Amount
df['Total_amount']= df['Unit_Price']* df['Quantity']

### Filling missing values
df['Item_Name'] = df['Item_Name'].replace(r'^\s*$', np.nan, regex=True).fillna('UNKNOWN_ITEM')

df['Item_Name'] = list(map (lambda x: (str(x).strip()).title(),df['Item_Name']))

df['Department'] = list(map (lambda x: str(x).upper(),df['Department']))

df['Bar_Code'] = list(map (lambda x: str(x).strip('.0'),df['Bar_Code']))

df['Department'] = list(map (lambda x: "UNKNOWN_DEPT" if x=='NAN' else x,df['Department']))

df.drop(df[(df['Unit_Price']==0)&(df['Total_amount']==0)].index, inplace=True)

### Date Range Validation
Step 12: Display earliest, latest, and range of transaction dates in the dataset


In [ ]:
print("Earliest date:", df['Sales_RealDate'].min())
print("Latest date:", df['Sales_RealDate'].max())
print("Date range:", df['Sales_RealDate'].max() - df['Sales_RealDate'].min())

# SKU analytics 

1. Total Sales (Value & Volume)


In [ ]:
# Value (Revenue)
total_sales_value = df["Total_amount"].sum()

# Volume (Units sold)
total_sales_volume = df["Quantity"].sum()

print("Total Sales Value:", round(total_sales_value,2))
print("Total Sales Volume:", total_sales_volume)

2. Sales per Department

In [ ]:
sales_by_department = df.groupby("Department").agg({
    "Total_amount": "sum",
    "Quantity": "sum"
}).reset_index()

print(sales_by_department)

3. Average Selling Price (ASP)
- ASP is Total Sales ÷ Quantity.

In [ ]:
# Overall ASP (Average Selling Price)
asp_overall = df["Total_amount"].sum() / df["Quantity"].sum()

# ASP per Item using groupby().agg() to avoid warnings
asp_item = df.groupby("Item_Name").agg({
    "Total_amount": "sum",
    "Quantity": "sum"
}).assign(ASP=lambda x: x["Total_amount"] / x["Quantity"]).reset_index()

# Keep only necessary columns
asp_item = asp_item[["Item_Name", "ASP"]]

# Output
print("Overall ASP:", asp_overall)
print(asp_item.head(10))


In [ ]:
asp_sorted = asp_item.sort_values("ASP", ascending=False)

In [ ]:
plt.figure(figsize=(10,5))
### Top 10 SKUs (top 10 row from data) selected for Vizualization
sns.barplot(x="Item_Name", y="ASP", data=asp_sorted.head(20))

plt.title("Average Selling Price per SKU", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Average Selling Price")
plt.xlabel("SKU")
plt.tight_layout()
plt.show()

4. Contribution to Total Sales (%)

- How much each SKU/Item contributes to total sales.

In [ ]:
# Total sales for denominator
total_sales = df["Total_amount"].sum()

# Contribution per Item
contribution_item = df.groupby("Item_Name")["Total_amount"].sum().reset_index()
contribution_item["Contribution_%"] = (contribution_item["Total_amount"] / total_sales) * 100

print(contribution_item.sort_values("Contribution_%", ascending=False).head(10))

5. Order Frequency per SKU
- Number of distinct orders where SKU was present.

In [ ]:
# Order frequency per SKU (unique invoices)
order_freq = df.groupby("Item_Name")["Invoice_id"].nunique().reset_index()
order_freq.rename(columns={"Invoice_id": "Order_Frequency"}, inplace=True)

print(order_freq.head())

## Top SKUs

Step 13: Top SKUs/Items Based on Total Sales Amount ('Unit_Price' x 'Quantity')  and Total Quantity Sold

In [ ]:
## Group items and calculate total sales for each group
item_table = df.groupby(['Item_Name'], as_index=False).agg(total_amount=('Total_amount', 'sum'))
## Sort items by total sales amount in descending order to highlight top SKUs
item_table = item_table.sort_values('total_amount',ascending=False).reset_index(drop=True)
## Display the output table listing SKUs by total sales amount
item_table

Bar Chart – Total Sales (Value)

In [ ]:
plt.figure(figsize=(10,5))
### Top 10 SKUs (top 10 row from data) selected for Vizualization
sns.barplot(x="Item_Name", y="total_amount", data=item_table.sort_values("total_amount", ascending=False)[:10])

plt.title("Total Sales Value by SKU", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Sales Value")
plt.xlabel("SKU")
plt.tight_layout()
plt.show()

In [ ]:
## Group items and calculate total Quantity sold for each group
item_table1 = df.groupby(['Item_Name'], as_index=False).agg(total_qty=('Quantity', 'sum'))
## Sort items by total Quantity sold in descending order to highlight top SKUs
item_table1 = item_table1.sort_values('total_qty',ascending=False).reset_index(drop=True)
## Display the output table listing SKUs by total Quantity sold
item_table1

Bar Chart – Total Sales (Volume)

In [ ]:
plt.figure(figsize=(10,5))
### Top 10 SKUs (top 10 row from data) selected for Vizualization
sns.barplot(x="Item_Name", y="total_qty", data=item_table1.sort_values("total_qty", ascending=False)[:10])

plt.title("Total Sales Volume by SKU", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Units Sold")
plt.xlabel("SKU")
plt.tight_layout()
plt.show()

### 80-20 Analysis and Grouping
Step 14: Bucket items based on sales deciles for visualization and decision making

In [ ]:
## Number of group want to show in decile table 
group_number = 10

## Creating decile groups 
data_cut = pd.DataFrame()
data_cut['item'] = item_table['Item_Name']
data_cut['amount'] = pd.to_numeric(item_table.total_amount)


data_cut['bucket'] = pd.qcut(data_cut['amount'], group_number)
grouped = data_cut.groupby('bucket', as_index = False,observed=False)

### Output Tables and Visuals
Step 15: Generate cumulative summaries and display final tables for the decile analysis


In [ ]:
### Create an empty DataFrame to hold the decile analysis summary statistics

waterfall_table = pd.DataFrame()
waterfall_table['Decile'] = [i for i in range (group_number, 0, -1)]

# Populate it with calculated fields including decile number, minimum and maximum sales amounts, 
# total sales amount, and item count for each decile group as derived from the grouped data
waterfall_table['min_amount'] = grouped.min()['amount']
waterfall_table['max_amount'] = grouped.max()['amount']
waterfall_table['tot_amount'] = grouped.sum()['amount']
waterfall_table['item_count'] = grouped.count()['item']

In [ ]:
## Step 16: Sort each decile group in descending order by minimum item value 
waterfall_table = waterfall_table.sort_values(by="min_amount", ascending=False).reset_index(drop = True)

### Calculate cumulative total amount and cumulative count across the sorted decile groups
waterfall_table['cumu_tot_amount'] = waterfall_table['tot_amount'].cumsum()
waterfall_table['cumu_count'] = waterfall_table['item_count'].cumsum()

In [ ]:
## Sort each decile group in descending order by minimum item value
waterfall_table = waterfall_table.sort_values(by="min_amount", ascending=False).reset_index(drop=True)  # Highest-value groups appear first for clearer cumulative trend analysis

## Calculate the cumulative total amount across all groups, building a running sum for visualization
waterfall_table['cumu_tot_amount'] = waterfall_table['tot_amount'].cumsum()  # Shows how total sales accumulate group by group

## Calculate the running total count of items sold for each group, useful for 80-20 and Pareto analysis
waterfall_table['cumu_count'] = waterfall_table['item_count'].cumsum()  


In [ ]:
### Step 17: Calculate cumulative percentages of total amount and item count relative to their respective overall sums, 
### rounding to 2 and 0 decimal places for clarity in reporting
waterfall_table['%cumu_tot_amount'] =  round((waterfall_table['cumu_tot_amount']/waterfall_table['tot_amount'].sum())*100,2)
waterfall_table['%cumu_count'] =  round((waterfall_table['cumu_count']/waterfall_table['item_count'].sum())*100,0)

Step 18: Extract key cumulative metrics from the waterfall table for summary display -
count of items, cumulative percentage of total amount, and cumulative percentage of item count
 

In [ ]:
full_table = waterfall_table[['Decile','cumu_count','%cumu_tot_amount','%cumu_count']]
full_table

Pareto (80-20) Line Plot

In [ ]:
fig, ax1 = plt.subplots(figsize=(6,6))

# Bar chart for sales
ax1.plot(full_table['Decile'], full_table["%cumu_count"], color="darkblue", marker="o")
ax1.set_ylabel("Cumulative % of Item Count", color="darkblue")

# Cumulative % line (Pareto)
ax2 = ax1.twinx()
ax2.plot(full_table['Decile'], full_table["%cumu_tot_amount"], color="red", marker="o")
ax2.axhline(80, color="green", linestyle="--")  # 80% threshold line
ax2.set_ylabel("Cumulative % of Total Sales Amount", color="red")

# Add values on cumulative % points
for i, val in enumerate(full_table["%cumu_tot_amount"]):
    ax2.text(i, val+1, f"{val:.1f}%", color="red", fontsize=8, ha="left")

# Titles and formatting
plt.title("Pareto (80-20) Analysis of SKU Sales")
ax1.set_xlabel("Decile")
plt.xticks(rotation=90)
plt.show()

Step 19: Select the specified top group (e.g., top 2) from the waterfall summary table showing cumulative counts and percentages to analyze key contributors

In [ ]:
top_group = 2
table = waterfall_table[['cumu_count','%cumu_tot_amount','%cumu_count']].iloc[top_group-1:top_group]
table

Step 20: This extracts the highest-ranked SKUs with their Total Sales Amount under the top group (e.g. 20 % cumulative count) selected above 

In [ ]:
toplist = item_table.iloc[:table['cumu_count'].iloc[0]]
toplist

### Item of the Month and Top Items
Step : Identify top items per month and quarter for business insights

In [ ]:
## How many items to show in table
top_items = 10

df1 = df.groupby(['Year_of_purchase','Month_of_purchase','Month_year','Item_Name'], as_index=False).agg(total_sales = ('Total_amount', 'sum')).sort_values(by="total_sales", ascending=False).reset_index(drop = True)

# for top selling items 
df2 = df1.groupby(["Year_of_purchase","Month_of_purchase","Month_year"], as_index=False).nth(list(range(top_items)))
top10 = df2.sort_values(by=["Year_of_purchase","Month_of_purchase"])

In [ ]:
top10

### Conclusion and Learning Objectives
- This workbook equips practical skills for retail data handling:
- Reading and cleaning data sets with pandas
- Feature engineering for time-based analysis
- Identifying trends, top products, and seasonal effects in sales
- Applying Pareto and decile analysis for inventory control
- Experiment further with different groupings/filters to deepen insight!


# End of Workbook